In [ ]:
HopsworksProjectName = "TaxiDemandPrediction"

In [ ]:
import os
import sys
sys.path.append("../")

import APIKey

HopsworksAPIKey = APIKey.HOPSWORKSAPIKEY

# Fetching Historical Data from 2022 Onwards from the TLC Trip Data 

In [ ]:
from datetime import datetime 
import pandas as pd
sys.path.append("../src/")
import data

StartYear = 2022
EndYear = datetime.now().year
print(f"Downloading Data from Year {StartYear} to Year {EndYear}")

Rides = pd.DataFrame()

for y in range(StartYear, EndYear+1):
    
    #Download one whole Year worth of Data
    RidesOneWholeYear = data.LoadRawData(y)
    
    #Append Rows
    Rides = pd.concat([Rides, RidesOneWholeYear])

In [ ]:
TS_Data = data.TransformRawDataIntoTSData(Rides)

In [ ]:
TS_Data.rename(columns = {"PickupHour":"pickup_hour", "NumOfRides":"numrides", "PickupLocationID":"pickup_location_id"}, inplace = True)

In [ ]:
import hopsworks

In [ ]:
Project = hopsworks.login(project = HopsworksProjectName, api_key_value = HopsworksAPIKey)

In [ ]:
FeatureStore = Project.get_feature_store()

In [ ]:
FeatureGroupName = "ts_hourly_featuregroup"
FeatureGroupVersion = 1

In [ ]:
FeatureGroup = FeatureStore.get_or_create_feature_group(name = FeatureGroupName,
                                                        version = FeatureGroupVersion,
                                                        description = "TimeSeries Data at Hourly Frequency",
                                                        primary_key = ["pickup_location_id", "pickup_hour"],
                                                        event_time = "pickup_hour"
                                                       )

In [ ]:
FeatureGroup.insert(TS_Data, write_options = {"wait_for_job":False})